### Import Library dan CSV

In [1]:
import pandas as pd
import re
import string

from indoNLP.preprocessing import replace_word_elongation, replace_slang

In [47]:
angerData = pd.read_csv('./data/AngerData.csv', delimiter='\t')
fearData = pd.read_csv('./data/FearData.csv', delimiter='\t')
joyData = pd.read_csv('./data/JoyData.csv', delimiter='\t')
loveData = pd.read_csv('./data/LoveData.csv', delimiter='\t')
sadData = pd.read_csv('./data/SadData.csv', delimiter='\t')\
    
#gabungkan data
df = pd.concat([angerData, fearData, joyData, loveData, sadData], ignore_index=True)
df = df.sample(frac=1).reset_index(drop=True) #shuffle data

df.head()

,Tweet,Label
0,"ini segala keresahan gua udah mau ilang nihh, ...",Anger
1,rindumu sajak sentuhan. kalimatku pernah pucat...,Sad
2,lu tau seberapa gue keselnya? di shop gak ada ...,Anger
3,wkwkwk takut bgt digigit lgi,Fear
4,bangun tidur denger dua kabar ga enakin tuh se...,Sad


In [24]:
# Mengubag nama label menjadi bahasa Indonesia
label_mapping = {
    'Anger': 'Marah',
    'Fear': 'Takut',
    'Joy': 'Gembira',
    'Love': 'Cinta',
    'Sad': 'Sedih'
}
df['Label'] = df['Label'].map(label_mapping)
df.head()

,Tweet,Label
0,lho kirain fest hahahha :'),Takut
1,"berawal dari sahabat eh jadi pacar wkwk, terus...",Cinta
2,"iya cakep, emosi nya dapet banget pagi pagi",Marah
3,"adek w ldks sm pengukuhan osis lewat online, g...",Gembira
4,kak:( sumpah deh kenapa diksinya bagus banget ...,Cinta


In [35]:
with pd.option_context('display.max_colwidth', None):
    print(df.iloc[4])

Tweet    kak:( sumpah deh kenapa diksinya bagus banget gue suka banget anjir implisit yg kaya gini, astaga asli bagus banget
Label                                                                                                                  Cinta
Name: 4, dtype: object


### Cleaning Data

In [ ]:
def cleanDataframe(df):
    print('Missing Values:', df.isnull().sum())
    print('Duplicates:', df.duplicated().sum())

    df_cleaned = df.dropna(subset='Tweet')
    df_cleaned = df_cleaned.drop_duplicates(keep='first')

    print('\nMissing values after cleaning:', df_cleaned.isnull().sum())
    print('Duplicates after cleaning:', df_cleaned.duplicated().sum())
    return df_cleaned




def cleaningText(text):
    # Membersihkan tanda tanda sisa medsos
    text = re.sub(r'@[A-Za-z0-9]+', '', text)  # Menghapus Mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text)  # Menghapus Hashtag
    text = re.sub(r'RT[\s]', '', text)  # menghapus RT
    text = re.sub(r"http\S+", '', text)  # menghapus link

    # Pembersihan Karakter
    
    text = text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))# Mengganti tanda baca dengan spasi (alih-alih menghapus)
    text = re.sub(r'\d+', '', text)  # Menghapus angka
    text = text.replace('\n', ' ')  # Mengganti garis baru dengan spasi
    return text


kamusCleaning = pd.read_csv('kamus.csv', encoding='utf-8',
                            delimiter=',', header=None, names=['slang', 'formal'])
cleaningDict = dict(zip(kamusCleaning.slang, kamusCleaning.formal))


def applyCleaningDict(text):
    # Membersihkan kata yang tidak terbersihkan dari kamus IndoNLP dari kamus tambahan
    words = text.split()
    cleaned_words = [cleaningDict[word]
                     if word in cleaningDict else word for word in words]
    cleaned_text = ' '.join(cleaned_words)
    return cleaned_text

def normalizeText(text):
    text = text.strip(' ')  # Menghapus karakter spasi dari kiri dan kanan teks
    text = text.lower()  # mengubah semua karakter dalam teks menjadi huruf kecil
    text = re.sub(r'(.)\1+', r'\1\1', text) # Menghapus text elongation (library indoNLP)
    text = replace_word_elongation(text) # Menghapus huruf berlebih di belakang
    text = replace_slang(text)  # Menghapus slangwords dari kamus IndoNLP
    text = applyCleaningDict(text) # Membersihkan kata yang tidak terbersihkan dari kamus IndoNLP dari kamus tambahan
    text = re.sub(r'\s+', ' ', text) # Mengganti multiple spasi dengan satu spasi
    
    return text

In [26]:
df_cleaned = cleanDataframe(df)

Missing Values: Tweet    0
Label    0
dtype: int64
Duplicates: 42

Missing values after cleaning: Tweet    0
Label    0
dtype: int64
Duplicates after cleaning: 0


In [32]:
df_cleaned['Tweet'] = df_cleaned['Tweet'].apply(cleaningText)
# Print 1 sample not truncated
with pd.option_context('display.max_colwidth', None):
    print(df_cleaned.iloc[4])


Tweet    kak   sumpah deh kenapa diksinya bagus banget gue suka banget anjir implisit yg kaya gini  astaga asli bagus banget
Label                                                                                                                  Cinta
Name: 4, dtype: object


In [40]:
df_normalized = df_cleaned.copy()
df_normalized['Tweet'] = df_normalized['Tweet'].apply(normalizeText)
# Print 1 sample not truncated
with pd.option_context('display.max_colwidth', None):
    print(df_normalized.iloc[4])

Tweet    kak sumpah deh kenapa diksinya bagus sekali saya suka sekali anjir implisit yang mirip begini astaga asli bagus sekali 
Label                                                                                                                      Cinta
Name: 4, dtype: object


In [7]:
print(df_cleaned['Label'].value_counts())

Label
Gembira    1254
Marah      1043
Sedih       995
Takut       906
Cinta       754
Name: count, dtype: int64


### Export Menjadi CSV Baru

In [8]:
# Export
df_cleaned.to_csv('./data/data_cleaned.csv', index=False)